Bazham Khanatayev \
Data 512 HW_2 \
10.15.2023 \
The purpose of this notebook is to prepare the article_data.csv for the ORES api. The output of this notebook is the article_data_clean.csv

In [1]:
import pandas as pd

In [29]:
# Read the CSV into a DataFrame
df_raw = pd.read_csv("article_data.csv")

The following blocks are for some basic data exploration. We want to understand the commas in the Article Titles so that we can most effectively parse the state out without losing data.

In [3]:
df_raw.shape

(21519, 3)

In [ ]:
# Check how many parts are produced by the split operation for each entry
splits = df_raw['Article Title'].str.split(',', expand=True)

We want to isolate the rows were the titles do not split into exactly three parts.These rows will require some more processing to retrieve the state.

In [ ]:
# Print entries that don't split into exactly two parts
problematic_entries = splits[splits.iloc[:, 2].notna() | splits[1].isna()]
print(problematic_entries)

In [5]:
if problematic_entries.empty:
    df_raw[['City', 'State']] = splits[[0, 1]]
    df_raw['City'] = df_raw['City'].str.strip()
    df_raw['State'] = df_raw['State'].str.strip()
    print(df_raw.head())
else:
    print("There are entries in the 'Article Title' column that don't split into exactly two parts.")

                               0                   1               2
11                      Anderson   Lauderdale County         Alabama
134                    Evergreen      Conecuh County         Alabama
382                       Shiloh       DeKalb County         Alabama
601                    Utqiaġvik                None            None
1015                      Midway   Hot Spring County        Arkansas
...                          ...                 ...             ...
20254  Wikipedia:Citation needed                None            None
20879                    Seattle                None            None
21031            Pleasant Valley       Marion County   West Virginia
21285                   Franklin    Milwaukee County       Wisconsin
21332                  Milwaukee                None            None

[2030 rows x 3 columns]
There are entries in the 'Article Title' column that don't split into exactly two parts.


In [ ]:
# Split the 'Article Title' column
splits = df_raw['Article Title'].str.split(',', expand=True)

In [ ]:
# Identify the problematic entries
problematic_entries = df_raw[splits.iloc[:, 2].notna() | splits[1].isna()]

In [ ]:
# Store problematic 'Article Title' values in a list
problematic_titles = problematic_entries['Article Title'].tolist()

In [6]:
# Print the problematic titles
for title in problematic_titles:
    print(title)

Anderson, Lauderdale County, Alabama
Evergreen, Conecuh County, Alabama
Shiloh, DeKalb County, Alabama
Utqiaġvik
Midway, Hot Spring County, Arkansas
Oak Grove, Carroll County, Arkansas
Salem, Fulton County, Arkansas
Southside, Independence County, Arkansas
Sulphur Springs, Benton County, Arkansas
Live Oak, Sutter County, California
Los Angeles
San Diego
San Francisco
2020 United States census
2010 United States census
Coal Creek, Fremont County, Colorado
Denver
County (United States)
Edgewater, Volusia County, Florida
Lake Butler, Union County, Florida
Miami
Midway, Gadsden County, Florida
Sweetwater, Miami-Dade County, Florida
Atlanta
Buckhead, Morgan County, Georgia
Echols County
Georgetown, Quitman County, Georgia
Mountain Park, Fulton County, Georgia
Pine Mountain, Harris County, Georgia
Kailua, Honolulu County, Hawaii
Kailua, Hawaii County, Hawaii
Waimea, Hawaii County, Hawaii
Keokea, Maui County, Hawaii
Waimea, Kauai County, Hawaii
Hāʻena, Kauai County, Hawaii
New Salem, Pike Cou

Rockland Township, Berks County, Pennsylvania
Green Township, Indiana County, Pennsylvania
Hempfield Township, Mercer County, Pennsylvania
Jefferson Township, Lackawanna County, Pennsylvania
Clinton Township, Lycoming County, Pennsylvania
Paupack Township, Wayne County, Pennsylvania
Alsace Township, Berks County, Pennsylvania
North Manheim Township, Schuylkill County, Pennsylvania
Greenwich Township, Berks County, Pennsylvania
Point Township, Northumberland County, Pennsylvania
Sadsbury Township, Chester County, Pennsylvania
Colerain Township, Lancaster County, Pennsylvania
Hanover Township, Beaver County, Pennsylvania
Mount Joy Township, Adams County, Pennsylvania
Oley Township, Berks County, Pennsylvania
Jenks Township, Forest County, Pennsylvania
Jackson Township, Butler County, Pennsylvania
Pennsbury Township, Chester County, Pennsylvania
Damascus Township, Wayne County, Pennsylvania
Tilden Township, Berks County, Pennsylvania
Fox Township, Elk County, Pennsylvania
Portage Township

Kline Township, Schuylkill County, Pennsylvania
Sterling Township, Wayne County, Pennsylvania
Worth Township, Butler County, Pennsylvania
Ransom Township, Lackawanna County, Pennsylvania
Harford Township, Susquehanna County, Pennsylvania
Huston Township, Clearfield County, Pennsylvania
North Mahoning Township, Indiana County, Pennsylvania
South Newton Township, Cumberland County, Pennsylvania
Warsaw Township, Jefferson County, Pennsylvania
Fishing Creek Township, Columbia County, Pennsylvania
Mahoning Township, Armstrong County, Pennsylvania
Locust Township, Columbia County, Pennsylvania
Jefferson Township, Somerset County, Pennsylvania
Jackson Township, Snyder County, Pennsylvania
Juniata Township, Perry County, Pennsylvania
Union Township, Centre County, Pennsylvania
Liberty Township, Mercer County, Pennsylvania
Little Beaver Township, Lawrence County, Pennsylvania
Dyberry Township, Wayne County, Pennsylvania
Washington Township, Wyoming County, Pennsylvania
East Finley Township, Was

Jefferson, Greene County, Pennsylvania
Polk Township, Jefferson County, Pennsylvania
Liberty, Tioga County, Pennsylvania
West Abington Township, Lackawanna County, Pennsylvania
Gilmore Township, Greene County, Pennsylvania
Foster Township, Schuylkill County, Pennsylvania
Mayberry Township, Montour County, Pennsylvania
Barnett Township, Jefferson County, Pennsylvania
Hopewell, Bedford County, Pennsylvania
Overton Township, Bradford County, Pennsylvania
Spruce Creek Township, Huntingdon County, Pennsylvania
Reed Township, Dauphin County, Pennsylvania
Lausanne Township, Carbon County, Pennsylvania
Rush Township, Dauphin County, Pennsylvania
Spring Creek Township, Elk County, Pennsylvania
Portage Township, Potter County, Pennsylvania
Gray Township, Greene County, Pennsylvania
Centerville, Crawford County, Pennsylvania
Cherry Grove Township, Warren County, Pennsylvania
Duncan Township, Tioga County, Pennsylvania
North Branch Township, Wyoming County, Pennsylvania
Benezette Township, Elk Cou

In [7]:
len(problematic_titles)

2030

Now that we have an idea of how the data is structed. It makes to split the titles by everything to the left of the final comma and everything to the right of the final comma.

In [8]:
def split_article_title(row):
    parts = row['Article Title'].split(',')
    if len(parts) == 3:
        city = parts[0] + ',' + parts[1]
        state = parts[2]
    elif len(parts) == 2:
        city = parts[0]
        state = parts[1]
    else:
        city = state = None  # I can handle additional cases here if I need to
    return pd.Series([city.strip(), state.strip()], index=['City', 'State'])


In [9]:
# Update the problematic_titles list to exclude three-part titles
filtered_titles = []
for title in problematic_titles:
    if title.count(',') != 2:  # Keep titles that are not three-part
        filtered_titles.append(title)

In [10]:
# Now, filtered_titles contains the titles we want to remove from df_raw
df_clean = df_raw[~df_raw['Article Title'].isin(filtered_titles)].copy()

In [11]:
df_clean.shape

(21486, 3)

In [12]:
# Now, run the function we defined earlier to split the titles
df_clean[['City', 'State']] = df_clean.apply(split_article_title, axis=1)

In [14]:
filtered_titles

['Utqiaġvik',
 'Los Angeles',
 'San Diego',
 'San Francisco',
 '2020 United States census',
 '2010 United States census',
 'Denver',
 'County (United States)',
 'Miami',
 'Atlanta',
 'Echols County',
 'Indianapolis',
 'New Orleans',
 'Boston',
 'Nantucket',
 'Detroit',
 'Minneapolis',
 'St. Louis',
 'Las Vegas',
 'Population',
 'Square mile',
 'New York City',
 'Federal Information Processing Standards',
 'American National Standards Institute',
 'Geographic Names Information System',
 'Oklahoma City',
 'Philadelphia',
 'Pittsburgh',
 'Grand Divisions of Tennessee',
 'Salt Lake City',
 'Wikipedia:Citation needed',
 'Seattle',
 'Milwaukee']

In [ ]:
# Add 'Kauai' and 'Boston' and 'St. Louis' and 'Syracuse' to the filtered_titles list
filtered_titles.extend(['Wailua, Kauai', 'Hyde Park, Boston', 'St. Louis', 'Syracuse'])

In [ ]:
# Extract the rows from df_raw corresponding to filtered_titles
problematic_rows = df_raw[df_raw['Article Title'].isin(filtered_titles)]

In [16]:
# Output to a CSV
problematic_rows.to_csv('problematic_titles_log.csv', index=False)

In [22]:
# remove these rows from df_clean
df_clean = df_clean[~df_clean['State'].isin(['Kauai', 'Boston', 'St. Louis', 'Syracuse'])]


Check for duplicate rows

In [24]:
df_clean[df_clean.duplicated()]

,Article Title,URL,Revision ID,City,State


In [ ]:
# Create a copy of df_clean
df_output = df_clean.copy()

In [ ]:
# Drop the 'City' and 'State' columns
df_output.drop(columns=['City', 'State'], inplace=True)

In [25]:
# Export df_output to a CSV file
df_output.to_csv('article_data_clean.csv', index=False)

Sanity Check

In [28]:
df_output.shape

(21483, 3)